In [7]:
import numpy as np
import pandas as pd
from pandas.plotting import table
import sqlite3
import matplotlib.pyplot as plt
import datetime as dt
from bs4 import BeautifulSoup as BS
import requests
import re

In [2]:
conn = sqlite3.connect('/home/josh/Documents/dsi/caps/cap2/pot_data/database.sqlite')

In [3]:
c = conn.cursor()

In [4]:
tab1 = '''select * from Player'''
c.execute(tab1)

In [5]:
play_data = c.fetchall()

In [6]:
df_player = pd.read_sql('select * from Player',con=conn)
df_player.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [33]:
df_phead = df_player.head()
from bokeh.io import export_png, export_svgs
from bokeh.models import ColumnDataSource, DataTable, TableColumn

def save_df_as_image(df, path):
    source = ColumnDataSource(df)
    df_columns = [df.index.name]
    df_columns.extend(df.columns.values)
    columns_for_table=[]
    for column in df_columns:
        columns_for_table.append(TableColumn(field=column, title=column))
    data_table = DataTable(source=source,columns=columns_for_table,width=1000,index_position=None)
    export_png(data_table, filename = path)

save_df_as_image(df_phead,'../images/df_phead.png')

In [7]:
df_player.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  11060 non-null  int64  
 1   player_api_id       11060 non-null  int64  
 2   player_name         11060 non-null  object 
 3   player_fifa_api_id  11060 non-null  int64  
 4   birthday            11060 non-null  object 
 5   height              11060 non-null  float64
 6   weight              11060 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 605.0+ KB


In [11]:
df_play = df_player.copy()

In [12]:
df_play.drop(['id','player_api_id'],axis=1,inplace=True)

In [10]:
df_play.head()

,player_name,player_fifa_api_id,birthday,height,weight
0,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [11]:
df_play.birthday = pd.to_datetime(df_play.birthday,format='%Y/%m/%d')

In [12]:
df_play.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   player_name         11060 non-null  object        
 1   player_fifa_api_id  11060 non-null  int64         
 2   birthday            11060 non-null  datetime64[ns]
 3   height              11060 non-null  float64       
 4   weight              11060 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 432.2+ KB


In [13]:
df_play['age'] = [abs(x - pd.to_datetime(dt.date.today())) for x in df_play.birthday]

In [15]:
df_play.age = df_play.age/dt.timedelta(days=365)

In [16]:
df_play.head()

,player_name,player_fifa_api_id,birthday,height,weight,age
0,Aaron Appindangoye,218353,1992-02-29,182.88,187,28.501370
1,Aaron Cresswell,189615,1989-12-15,170.18,146,30.709589
2,Aaron Doran,186170,1991-05-13,170.18,163,29.301370
3,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548
4,Aaron Hughes,17725,1979-11-08,182.88,154,40.819178


In [17]:
conn

In [13]:
df_player_atts = pd.read_sql('select * from Player_Attributes',con=conn)
df_player_atts.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [38]:
df_ahead = df_player_atts.head()
def save_df_as_image(df, path):
    source = ColumnDataSource(df)
    df_columns = [df.index.name]
    df_columns.extend(df.columns.values)
    columns_for_table=[]
    for column in df_columns:
        columns_for_table.append(TableColumn(field=column, title=column))
    data_table = DataTable(source=source,columns=columns_for_table,width=1300,index_position=None)
    export_png(data_table, filename = path)

save_df_as_image(df_ahead.iloc[:,:11],'../images/df_ahead.png')

In [19]:
df_patts = df_player_atts.copy()

In [20]:
df_patts.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

In [21]:
df_patts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

In [22]:
df_patts.date = pd.to_datetime(df_patts.date,format='%Y/%m/%d')

In [23]:
df_patts.drop(['id','player_api_id'],axis=1,inplace=True)

In [22]:
# overall_rating',
#        'potential', 'preferred_foot', 'attacking_work_rate',
#        'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
#        'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
#        'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
#        'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
#        'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
#        'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
#        'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
#        'gk_reflexes'],

In [24]:
df = pd.merge(df_play,df_patts,how='left',on='player_fifa_api_id')

In [25]:
df_33up = df[df.age >= 33]

In [186]:
df_33up.iloc[:25,:]

,player_name,player_fifa_api_id,birthday,height,weight,age,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
64,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2016-04-21,69.0,69.0,right,...,54.0,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0
65,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2015-10-16,69.0,69.0,right,...,54.0,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0
66,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2015-10-02,69.0,69.0,right,...,54.0,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0
67,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2015-09-21,69.0,69.0,right,...,54.0,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0
68,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2015-07-03,67.0,67.0,right,...,53.0,36.0,66.0,68.0,65.0,14.0,11.0,12.0,11.0,10.0
69,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2015-06-12,67.0,67.0,right,...,53.0,36.0,66.0,68.0,65.0,14.0,11.0,12.0,11.0,10.0
70,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2015-01-16,67.0,67.0,right,...,53.0,36.0,66.0,68.0,65.0,14.0,11.0,12.0,11.0,10.0
71,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2014-11-14,67.0,67.0,right,...,53.0,36.0,66.0,68.0,65.0,14.0,11.0,12.0,11.0,10.0
72,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2014-09-18,67.0,67.0,right,...,53.0,36.0,66.0,68.0,65.0,14.0,11.0,12.0,11.0,10.0
73,Aaron Galindo,140161,1982-05-08,182.88,198,38.320548,2014-06-06,67.0,67.0,right,...,54.0,37.0,67.0,69.0,66.0,14.0,11.0,12.0,11.0,10.0


In [27]:
pnames = df_33up.player_name.unique()
len(pnames)

5548

In [210]:
url = 'https://fbref.com/en/players/9076667f/all_comps/Aaron-Galindo-Stats---All-Competitions'
r = requests.get(url)
soup = BS(r.content,'html.parser')

In [211]:
h1s_ = soup.find_all('h1')

In [212]:
play_nm = [item.get_text(strip=True) for item in h1s_[0].select("span")][0]
play_nm

'Aarón Galindo'

In [213]:
p_strong = [item.next_sibling for item in soup.select("p strong")][:3]

In [108]:
p_strong

[None, ' DF (CB)\n\n\n  ', '\n']

In [214]:
p_pos = re.sub(r'\n\s+','',p_strong[1])
p_pos

' DF (CB)'

In [215]:
tabs = soup.find_all('table')
tbod = soup.find_all('tbody')

In [31]:
tbod

[<tbody>
 <tr id="stats" style="line-height: 1.3em"><th class="left" data-stat="season" scope="row">2003-2004</th><td class="center" data-stat="age">21</td><td class="left" data-stat="squad"><a href="/en/squads/632f1838/2003-2004/Cruz-Azul-Stats">Cruz Azul</a></td><td class="left" data-stat="country"><a href="/en/country/MEX/Mexico-Football"><span class="f-i" style="background-image:url('https://d2p3bygnnzw9w3.cloudfront.net/req/202008171/flags/mx-2003.svg'); ">mx</span></a> <a href="/en/country/MEX/Mexico-Football">MEX</a></td><td class="left" csk="1a.31" data-stat="comp_level"><span class="level">1.</span> <a href="/en/comps/31/120/2003-2004-Primera-Division-Stats">Primera División</a></td><td class="right group_start" data-stat="games">25</td><td class="right" data-stat="games_starts">24</td><td class="right" csk="2171" data-stat="minutes">2,171</td><td class="right group_start" data-stat="goals">2</td><td class="right" data-stat="assists">2</td><td class="right iz" data-stat="pens_

In [88]:
# tabs - not used
# galindo_rows = tabs[0].find_all('tr')
# galindo_rows

In [216]:
play_rows = tbod[0].find_all('tr')

[<tr class="over_header">
 <th aria-label="" class="over_header center" colspan="5" data-stat=""></th>
 <th aria-label="" class="over_header center group_start" colspan="3" data-stat="header_playing">Playing Time</th>
 <th aria-label="" class="over_header center group_start" colspan="6" data-stat="header_performance">Performance</th>
 <th aria-label="" class="over_header center group_start" colspan="5" data-stat="header_per90">Per 90 Minutes</th><th></th>
 </tr>,
 <tr>
 <th aria-label="Season" class="poptip center" data-stat="season" data-tip="&lt;strong&gt;Season&lt;/strong&gt;&lt;br&gt;Winter leagues generally run from August to May of the following year.&lt;br&gt;Summer leagues generally run from February to November of the same year." scope="col">Season</th>
 <th aria-label="Age at season start" class="poptip sort_default_asc center" data-stat="age" data-tip="&lt;strong&gt;Age at the season start for league play&lt;/strong&gt;&lt;br&gt;Given on August 1 for winter leagues&lt;br&gt;

In [217]:
data = []
y_played = []
for pr in play_rows:
    tds = pr.find_all('td')
    yrs = pr.find_all('th',attrs={'scope':'row'})
    yr = [y.text for y in yrs]
    row = [td_.text for td_ in tds]
    data.append(row)
    y_played.append(yr)

In [165]:
data

[['21',
  'Cruz Azul',
  'mx MEX',
  '1. Primera División',
  '25',
  '24',
  '2,171',
  '2',
  '2',
  '0',
  '0',
  '3',
  '0',
  '0.08',
  '0.08',
  '0.17',
  '0.08',
  '0.17',
  ''],
 ['22',
  'Cruz Azul',
  'mx MEX',
  '1. Primera División',
  '29',
  '28',
  '2,450',
  '4',
  '2',
  '0',
  '0',
  '9',
  '0',
  '0.15',
  '0.07',
  '0.22',
  '0.15',
  '0.22',
  ''],
 ['24',
  'Hércules',
  'es ESP',
  '2. Segunda División',
  '5',
  '5',
  '448',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  ''],
 ['24',
  'Grasshopper',
  'ch SUI',
  '1. Super League',
  '15',
  '15',
  '1,305',
  '2',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0.14',
  '0.00',
  '0.14',
  '0.14',
  '0.14',
  ''],
 ['25',
  'Eint Frankfurt',
  'de GER',
  '1. Bundesliga',
  '22',
  '21',
  '1,932',
  '0',
  '0',
  '0',
  '0',
  '2',
  '0',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'Matches'],
 ['26',
  'Eint Frankfurt',
  'de GER',
  '1. Bundesliga',
  '10',
  '1

In [218]:
tester = pd.DataFrame(data)
tester['player_name'] = [play_nm for _ in range(len(data))]
tester['position'] = [p_pos for _ in range(len(data))]
tester['year'] = y_played

In [219]:
tester.columns = ['age','club','country','division','matches_played','starts','minutes_played','goals',
                'assists','penalty_kicks','penalty_kick_attempts','yellow_cards','red_cards','goals_per_90',
               'assists_per_90','goals_and_assists_per_90','goals_minus_pk_per_90','goals+assists-pk_per_90',
               'matches','player_name','position','year']
tester

,age,club,country,division,matches_played,starts,minutes_played,goals,assists,penalty_kicks,...,red_cards,goals_per_90,assists_per_90,goals_and_assists_per_90,goals_minus_pk_per_90,goals+assists-pk_per_90,matches,player_name,position,year
0,21,Cruz Azul,mx MEX,1. Primera División,25,24,"2,171",2,2,0,...,0,0.08,0.08,0.17,0.08,0.17,,Aarón Galindo,DF (CB),[2003-2004]
1,22,Cruz Azul,mx MEX,1. Primera División,29,28,"2,450",4,2,0,...,0,0.15,0.07,0.22,0.15,0.22,,Aarón Galindo,DF (CB),[2004-2005]
2,24,Hércules,es ESP,2. Segunda División,5,5,448,0,0,0,...,1,0.00,0.00,0.00,0.00,0.00,,Aarón Galindo,DF (CB),[2006-2007]
3,24,Grasshopper,ch SUI,1. Super League,15,15,"1,305",2,0,0,...,0,0.14,0.00,0.14,0.14,0.14,,Aarón Galindo,DF (CB),[2006-2007]
4,25,Eint Frankfurt,de GER,1. Bundesliga,22,21,"1,932",0,0,0,...,0,0.00,0.00,0.00,0.00,0.00,Matches,Aarón Galindo,DF (CB),[2007-2008]
5,26,Eint Frankfurt,de GER,1. Bundesliga,10,10,900,0,0,0,...,0,0.00,0.00,0.00,0.00,0.00,Matches,Aarón Galindo,DF (CB),[2008-2009]
6,26,Guadalajara,mx MEX,1. Primera División,11,11,949,1,1,0,...,0,0.09,0.09,0.19,0.09,0.19,Matches,Aarón Galindo,DF (CB),[2008-2009]
7,27,Guadalajara,mx MEX,1. Primera División,28,27,"2,307",2,0,0,...,1,0.08,0.00,0.08,0.08,0.08,,Aarón Galindo,DF (CB),[2009-2010]
8,29,Santos,mx MEX,1. Primera División,18,12,"1,252",2,0,0,...,1,0.14,0.00,0.14,0.14,0.14,,Aarón Galindo,DF (CB),[2011-2012]
9,30,Santos,mx MEX,1. Liga MX,16,14,"1,270",0,1,0,...,0,0.00,0.07,0.07,0.00,0.07,,Aarón Galindo,DF (CB),[2012-2013]


In [222]:
tester.insert(4,'league_position',0)
tester

,age,club,country,division,league_position,matches_played,starts,minutes_played,goals,assists,...,red_cards,goals_per_90,assists_per_90,goals_and_assists_per_90,goals_minus_pk_per_90,goals+assists-pk_per_90,matches,player_name,position,year
0,21,Cruz Azul,mx MEX,1. Primera División,0,25,24,"2,171",2,2,...,0,0.08,0.08,0.17,0.08,0.17,,Aarón Galindo,DF (CB),[2003-2004]
1,22,Cruz Azul,mx MEX,1. Primera División,0,29,28,"2,450",4,2,...,0,0.15,0.07,0.22,0.15,0.22,,Aarón Galindo,DF (CB),[2004-2005]
2,24,Hércules,es ESP,2. Segunda División,0,5,5,448,0,0,...,1,0.00,0.00,0.00,0.00,0.00,,Aarón Galindo,DF (CB),[2006-2007]
3,24,Grasshopper,ch SUI,1. Super League,0,15,15,"1,305",2,0,...,0,0.14,0.00,0.14,0.14,0.14,,Aarón Galindo,DF (CB),[2006-2007]
4,25,Eint Frankfurt,de GER,1. Bundesliga,0,22,21,"1,932",0,0,...,0,0.00,0.00,0.00,0.00,0.00,Matches,Aarón Galindo,DF (CB),[2007-2008]
5,26,Eint Frankfurt,de GER,1. Bundesliga,0,10,10,900,0,0,...,0,0.00,0.00,0.00,0.00,0.00,Matches,Aarón Galindo,DF (CB),[2008-2009]
6,26,Guadalajara,mx MEX,1. Primera División,0,11,11,949,1,1,...,0,0.09,0.09,0.19,0.09,0.19,Matches,Aarón Galindo,DF (CB),[2008-2009]
7,27,Guadalajara,mx MEX,1. Primera División,0,28,27,"2,307",2,0,...,1,0.08,0.00,0.08,0.08,0.08,,Aarón Galindo,DF (CB),[2009-2010]
8,29,Santos,mx MEX,1. Primera División,0,18,12,"1,252",2,0,...,1,0.14,0.00,0.14,0.14,0.14,,Aarón Galindo,DF (CB),[2011-2012]
9,30,Santos,mx MEX,1. Liga MX,0,16,14,"1,270",0,1,...,0,0.00,0.07,0.07,0.00,0.07,,Aarón Galindo,DF (CB),[2012-2013]


In [223]:
df_play_ann_stats = pd.DataFrame(data)
df_play_ann_stats['player_name'] = [play_nm for _ in range(len(data))]
df_play_ann_stats['position'] = [p_pos for _ in range(len(data))]
df_play_ann_stats['year'] = y_played

In [226]:
df_play_ann_stats.insert(4,'league_position',0)

In [227]:
annstatscols = ['age','club','country_playedin','division','league_position','matches_played','starts','minutes_played','goals',
                'assists','penalty_kicks','penalty_kick_attempts','yellow_cards','red_cards','goals_per_90',
               'assists_per_90','goals_and_assists_per_90','goals_minus_pk_per_90','goals+assists-pk_per_90',
               'matches','player_name','position','year']
df_play_ann_stats.columns = annstatscols

In [244]:
df_ann_stats = df_play_ann_stats.copy()
df_ann_stats

,age,club,country_playedin,division,league_position,matches_played,starts,minutes_played,goals,assists,...,red_cards,goals_per_90,assists_per_90,goals_and_assists_per_90,goals_minus_pk_per_90,goals+assists-pk_per_90,matches,player_name,position,year
0,21,Cruz Azul,mx MEX,1. Primera División,0,25,24,"2,171",2,2,...,0,0.08,0.08,0.17,0.08,0.17,,Aarón Galindo,DF (CB),[2003-2004]
1,22,Cruz Azul,mx MEX,1. Primera División,0,29,28,"2,450",4,2,...,0,0.15,0.07,0.22,0.15,0.22,,Aarón Galindo,DF (CB),[2004-2005]
2,24,Hércules,es ESP,2. Segunda División,0,5,5,448,0,0,...,1,0.00,0.00,0.00,0.00,0.00,,Aarón Galindo,DF (CB),[2006-2007]
3,24,Grasshopper,ch SUI,1. Super League,0,15,15,"1,305",2,0,...,0,0.14,0.00,0.14,0.14,0.14,,Aarón Galindo,DF (CB),[2006-2007]
4,25,Eint Frankfurt,de GER,1. Bundesliga,0,22,21,"1,932",0,0,...,0,0.00,0.00,0.00,0.00,0.00,Matches,Aarón Galindo,DF (CB),[2007-2008]
5,26,Eint Frankfurt,de GER,1. Bundesliga,0,10,10,900,0,0,...,0,0.00,0.00,0.00,0.00,0.00,Matches,Aarón Galindo,DF (CB),[2008-2009]
6,26,Guadalajara,mx MEX,1. Primera División,0,11,11,949,1,1,...,0,0.09,0.09,0.19,0.09,0.19,Matches,Aarón Galindo,DF (CB),[2008-2009]
7,27,Guadalajara,mx MEX,1. Primera División,0,28,27,"2,307",2,0,...,1,0.08,0.00,0.08,0.08,0.08,,Aarón Galindo,DF (CB),[2009-2010]
8,29,Santos,mx MEX,1. Primera División,0,18,12,"1,252",2,0,...,1,0.14,0.00,0.14,0.14,0.14,,Aarón Galindo,DF (CB),[2011-2012]
9,30,Santos,mx MEX,1. Liga MX,0,16,14,"1,270",0,1,...,0,0.00,0.07,0.07,0.00,0.07,,Aarón Galindo,DF (CB),[2012-2013]


In [27]:
player_lst = list(df_33up.player_name.unique())
len(player_lst)

5538

In [233]:
url = 'https://fbref.com/en/players/dccccccc/Aaron-Hughes'
r = requests.get(url)
soup = BS(r.content,'html.parser')

In [234]:
h1s_ = soup.find_all('h1')

In [235]:
play_nm = [item.get_text(strip=True) for item in h1s_[0].select("span")][0]
play_nm

'Aaron Hughes'

In [236]:
p_strong = [item.next_sibling for item in soup.select("p strong")][:2]
p_pos = re.sub(r'\n.','',p_strong[1])  #n\s+
p_pos

' DF (CB)▪\xa0 '

In [237]:
tbod = soup.find_all('tbody')
play_rows = tbod[0].find_all('tr')

In [238]:
data = []
y_played = []
for pr in play_rows:
    tds = pr.find_all('td')
    yrs = pr.find_all('th',attrs={'scope':'row'})
    yr = [y.text for y in yrs]
    row = [td_.text for td_ in tds]
    data.append(row)
    y_played.append(yr)

In [239]:
tester = pd.DataFrame(data)
tester['player_name'] = [play_nm for _ in range(len(data))]
tester['position'] = [p_pos for _ in range(len(data))]
tester['year'] = y_played

In [245]:
tester.head()

,age,club,countryin,division,league_position,matches_played,starts,minutes_played,goals,assists,...,red_cards,goals_per_90,assists_per_90,goals_and_assists_per_90,goals_minus_pk_per_90,goals+assists-pk_per_90,matches,player_name,position,year
0,18,Newcastle Utd,eng ENG,1. Premiership,13th,14,12,"1,105",0,1,...,0,0.00,0.08,0.08,0.00,0.08,Matches,Aaron Hughes,DF (CB)▪,[1998-1999]
1,19,Newcastle Utd,eng ENG,1. Premiership,11th,27,22,"1,993",2,2,...,0,0.09,0.09,0.18,0.09,0.18,Matches,Aaron Hughes,DF (CB)▪,[1999-2000]
2,20,Newcastle Utd,eng ENG,1. Premiership,11th,35,34,"2,994",0,0,...,0,0.00,0.00,0.00,0.00,0.00,Matches,Aaron Hughes,DF (CB)▪,[2000-2001]
3,21,Newcastle Utd,eng ENG,1. Premiership,4th,34,34,"3,042",0,4,...,0,0.00,0.12,0.12,0.00,0.12,Matches,Aaron Hughes,DF (CB)▪,[2001-2002]
4,22,Newcastle Utd,eng ENG,1. Premiership,3rd,35,35,"3,139",1,1,...,0,0.03,0.03,0.06,0.03,0.06,Matches,Aaron Hughes,DF (CB)▪,[2002-2003]


In [247]:
tester.columns = ['age','club','country_playedin','division','league_position','matches_played','starts',
                  'minutes_played','goals','assists','penalty_kicks','penalty_kick_attempts','yellow_cards',
                  'red_cards','goals_per_90', 'assists_per_90','goals_and_assists_per_90','goals_minus_pk_per_90',
                  'goals+assists-pk_per_90','matches','player_name','position','year']

In [248]:
df_ann_stats = pd.concat([df_ann_stats,tester],axis=0)
df_ann_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 0 to 22
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   age                       38 non-null     object
 1   club                      38 non-null     object
 2   country_playedin          38 non-null     object
 3   division                  38 non-null     object
 4   league_position           38 non-null     object
 5   matches_played            38 non-null     object
 6   starts                    38 non-null     object
 7   minutes_played            38 non-null     object
 8   goals                     38 non-null     object
 9   assists                   38 non-null     object
 10  penalty_kicks             38 non-null     object
 11  penalty_kick_attempts     38 non-null     object
 12  yellow_cards              38 non-null     object
 13  red_cards                 38 non-null     object
 14  goals_per_90              38

In [249]:
url = 'https://fbref.com/en/players/aa/'
r = requests.get(url)
soup = BS(r.content,'html.parser')

In [253]:
curr_link = soup.find_all('head')
curr_link[0]

<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202008171" rel="dns-prefetch"/>
<title>Football Players | FBref.com</title>
<meta content="Encyclopedia of World Football Players" name="Description"/>
<link href="https://fbref.com/en/players/aa/" rel="canonical"/>
<link href="https://fbref.com/pt/jogadores/aa/" hreflang="pt" rel="alternate"/>
<link href="https://fbref.com/en/players/aa/" hreflang="en" rel="alternate"/>
<link href="https://fbref.com/fr/joueurs/aa/" hreflang="fr" rel="alternate"/>
<link href="https://fbref.com/it/calciatori/aa/" hreflang="it" rel="alternate"/>
<link href="https://fbref.com/de/fussballspieler/aa/" hreflang="de" rel="alternate"/>
<link href="https://fbref.com/es/jugadores/aa/" hreflang="es" rel="alternate"/>
<!-- include:start ="/inc/klecko_header_fb.html_f" -->
<!-- no:cookie f

In [268]:
links = curr_link[0].find_all('link',attrs={'rel':'canonical'})
links

[<link href="https://fbref.com/en/players/aa/" rel="canonical"/>]

In [282]:
link = links[0]['href']
link

'https://fbref.com/en/players/aa/'

In [284]:
bod = soup.find_all('body')
bod

[<body class="fb">
 <div id="wrap">
 <div id="translations" role="navigation">
 <div class="links">
 <div class="welcome">Welcome, football fans from around the world!</div>
 <div><span class="use">Use FBref in: </span>
 <div class="langs">
 <a class="pt" href="https://fbref.com/pt/players/aa/" onclick="sr_set_trans_choice('pt');">Português</a>
          ·                        <a class="en" href="https://fbref.com/en/players/aa/" onclick="sr_set_trans_choice('en');">English</a>
          ·                        <a class="fr" href="https://fbref.com/fr/players/aa/" onclick="sr_set_trans_choice('fr');">Français</a>
          ·                        <a class="it" href="https://fbref.com/it/players/aa/" onclick="sr_set_trans_choice('it');">Italiano</a>
          ·                        <a class="de" href="https://fbref.com/de/players/aa/" onclick="sr_set_trans_choice('de');">Deutsch</a>
          ·                        <a class="es" href="https://fbref.com/es/players/aa/" onclick="s

In [290]:
div_list = bod[0].find_all('div',attrs={'class':'section_content'})
div_list[0]

<div class="section_content" id="div_1746074993">
<p><a href="/en/players/9c60f681/Ahmad-Aadi">Ahmad A'adi</a> <span class="f-i f-kw">kw</span> · 2014 · GK</p><p><a href="/en/players/ad713dff/Jamal-Aabbou"><strong>Jamal Aabbou</strong></a> <span class="f-i f-be">be</span> · 2018-2020 · FW · Lommel SK</p><p><a href="/en/players/c48b5529/Kim-Aabech"><strong>Kim Aabech</strong></a> <span class="f-i f-dk">dk</span> · 2007-2017 · MF · Lyngby, Aarhus, Horsens, Nordsjælland</p><p><a href="/en/players/d7ed844d/Kamilla-Aabel"><strong>Kamilla Aabel</strong></a> <span class="f-i f-no">no</span> · 2018-2020 · DF · Røa</p><p><a href="/en/players/13792bac/Rayane-Aabid"><strong>Rayane Aabid</strong></a> <span class="f-i f-fr">fr</span> · 2017-2019 · MF · Béziers, Paris FC</p><p><a href="/en/players/191084d1/Abdelilah-Aabiza"><strong>Abdelilah Aabiza</strong></a> <span class="f-i f-fr">fr</span> · 2017-2020 · FW · Bergerac Foot</p><p><a href="/en/players/cbe904fa/Hicham-Aaboubou">Hicham Aâboubou</a> <

In [291]:
a_list = div_list[0].find_all('a')
a_list

[<a href="/en/players/9c60f681/Ahmad-Aadi">Ahmad A'adi</a>,
 <a href="/en/players/ad713dff/Jamal-Aabbou"><strong>Jamal Aabbou</strong></a>,
 <a href="/en/players/c48b5529/Kim-Aabech"><strong>Kim Aabech</strong></a>,
 <a href="/en/players/d7ed844d/Kamilla-Aabel"><strong>Kamilla Aabel</strong></a>,
 <a href="/en/players/13792bac/Rayane-Aabid"><strong>Rayane Aabid</strong></a>,
 <a href="/en/players/191084d1/Abdelilah-Aabiza"><strong>Abdelilah Aabiza</strong></a>,
 <a href="/en/players/cbe904fa/Hicham-Aaboubou">Hicham Aâboubou</a>,
 <a href="/en/players/12648b72/Abubaker-Aadem">Abubaker Aadem</a>,
 <a href="/en/players/84129d72/Gerard-Aafjes">Gerard Aafjes</a>,
 <a href="/en/players/8f5fa9e1/Anna-Aahjem"><strong>Anna Aahjem</strong></a>,
 <a href="/en/players/f05ca35b/Faouzi-Aaish">Faouzi Aaish</a>,
 <a href="/en/players/0ade93c3/Abdelkader-Aajji"><strong>Abdelkader Aajji</strong></a>,
 <a href="/en/players/9a5b8092/Mohamed-Aakib"><strong>Mohamed Aakib</strong></a>,
 <a href="/en/players/

In [293]:
href_list = ['https://fbref.com' + x['href'] for x in a_list]
href_list

['https://fbref.com/en/players/9c60f681/Ahmad-Aadi',
 'https://fbref.com/en/players/ad713dff/Jamal-Aabbou',
 'https://fbref.com/en/players/c48b5529/Kim-Aabech',
 'https://fbref.com/en/players/d7ed844d/Kamilla-Aabel',
 'https://fbref.com/en/players/13792bac/Rayane-Aabid',
 'https://fbref.com/en/players/191084d1/Abdelilah-Aabiza',
 'https://fbref.com/en/players/cbe904fa/Hicham-Aaboubou',
 'https://fbref.com/en/players/12648b72/Abubaker-Aadem',
 'https://fbref.com/en/players/84129d72/Gerard-Aafjes',
 'https://fbref.com/en/players/8f5fa9e1/Anna-Aahjem',
 'https://fbref.com/en/players/f05ca35b/Faouzi-Aaish',
 'https://fbref.com/en/players/0ade93c3/Abdelkader-Aajji',
 'https://fbref.com/en/players/9a5b8092/Mohamed-Aakib',
 'https://fbref.com/en/players/4fe6659a/Andreas-Aalbu',
 'https://fbref.com/en/players/34793b4e/Levi-Aalders',
 'https://fbref.com/en/players/db4c2506/Redouan-Aalhoul',
 'https://fbref.com/en/players/690acb29/Henri-Aalto',
 'https://fbref.com/en/players/37804ea2/Iiro-Aalto'

In [296]:
play_nm_lst = [x.text for x in a_list]
play_nm_lst

["Ahmad A'adi",
 'Jamal Aabbou',
 'Kim Aabech',
 'Kamilla Aabel',
 'Rayane Aabid',
 'Abdelilah Aabiza',
 'Hicham Aâboubou',
 'Abubaker Aadem',
 'Gerard Aafjes',
 'Anna Aahjem',
 'Faouzi Aaish',
 'Abdelkader Aajji',
 'Mohamed Aakib',
 'Andreas Aalbu',
 'Levi Aalders',
 'Redouan Aalhoul',
 'Henri Aalto',
 'Iiro Aalto',
 'Jere Aalto',
 'Jussi Aalto',
 'Jani Aaltonen',
 'Mika Aaltonen',
 'Mika Aaltonen',
 'Patrick Aaltonen',
 'Marius Aam',
 'Ali Aamer',
 'Marianne Aamot Linaker',
 'Live Aandal',
 'Ismael Aaneba',
 'Markus Aanesland',
 'Vegard Aanestad',
 'Patrick van Aanholt',
 'Nabil Aankour',
 'Mads Aaquist',
 'Naïm Aarab',
 'Noura Al Aaraj',
 'Mika Ääritalo',
 'Signy Aarna',
 'Jari Aarne Vanhala',
 'Miko Aarne Virtanen',
 'Aldair Aaron Bravo',
 'Stephen Aaron Futcher',
 'Max Aarons',
 'Rolando Aarons',
 'Brenden Aaronson',
 'Paxten Aaronson',
 'Bjarte Aarsheim',
 'Fabiënne Aarts',
 'Sam Aarts',
 'Alexander Aas',
 'Christoffer Aasbak',
 'Tarjei Aase Omenås',
 'Daniel Aase',
 'Gunnar Aase

In [297]:
url = 'https://fbref.com/en/players/'
r = requests.get(url)
soup = BS(r.content,'html.parser')

In [299]:
bod = soup.find_all('body')
bod[0]

<body class="fb">
<div id="wrap">
<div id="translations" role="navigation">
<div class="links">
<div class="welcome">Welcome, football fans from around the world!</div>
<div><span class="use">Use FBref in: </span>
<div class="langs">
<a class="pt" href="https://fbref.com/pt/players/" onclick="sr_set_trans_choice('pt');">Português</a>
         ·                        <a class="en" href="https://fbref.com/en/players/" onclick="sr_set_trans_choice('en');">English</a>
         ·                        <a class="fr" href="https://fbref.com/fr/players/" onclick="sr_set_trans_choice('fr');">Français</a>
         ·                        <a class="it" href="https://fbref.com/it/players/" onclick="sr_set_trans_choice('it');">Italiano</a>
         ·                        <a class="de" href="https://fbref.com/de/players/" onclick="sr_set_trans_choice('de');">Deutsch</a>
         ·                        <a class="es" href="https://fbref.com/es/players/" onclick="sr_set_trans_choice('es');">Espa

In [301]:
div_list = bod[0].find_all('div',attrs={'class':'section_content'})
div_list[0]

<div class="section_content" id="div_7961720451">
<ul class="page_index"><li><span>A</span><div><a href="/en/players/aa/">Aa</a> · <a href="/en/players/ab/">Ab</a> · <a href="/en/players/ac/">Ac</a> · <a href="/en/players/ad/">Ad</a> · <a href="/en/players/ae/">Ae</a> · <a href="/en/players/af/">Af</a> · <a href="/en/players/ag/">Ag</a> · <a href="/en/players/ah/">Ah</a> · <a href="/en/players/ai/">Ai</a> · <a href="/en/players/aj/">Aj</a> · <a href="/en/players/ak/">Ak</a> · <a href="/en/players/al/">Al</a> · <a href="/en/players/am/">Am</a> · <a href="/en/players/an/">An</a> · <a href="/en/players/ao/">Ao</a> · <a href="/en/players/ap/">Ap</a> · <a href="/en/players/aq/">Aq</a> · <a href="/en/players/ar/">Ar</a> · <a href="/en/players/as/">As</a> · <a href="/en/players/at/">At</a> · <a href="/en/players/au/">Au</a> · <a href="/en/players/aw/">Aw</a> · <a href="/en/players/ax/">Ax</a> · <a href="/en/players/ay/">Ay</a> · <a href="/en/players/az/">Az</a></div></li><li><span>B</span><di

In [303]:
a_list = div_list[0].find_all('a')
a_list

[<a href="/en/players/aa/">Aa</a>,
 <a href="/en/players/ab/">Ab</a>,
 <a href="/en/players/ac/">Ac</a>,
 <a href="/en/players/ad/">Ad</a>,
 <a href="/en/players/ae/">Ae</a>,
 <a href="/en/players/af/">Af</a>,
 <a href="/en/players/ag/">Ag</a>,
 <a href="/en/players/ah/">Ah</a>,
 <a href="/en/players/ai/">Ai</a>,
 <a href="/en/players/aj/">Aj</a>,
 <a href="/en/players/ak/">Ak</a>,
 <a href="/en/players/al/">Al</a>,
 <a href="/en/players/am/">Am</a>,
 <a href="/en/players/an/">An</a>,
 <a href="/en/players/ao/">Ao</a>,
 <a href="/en/players/ap/">Ap</a>,
 <a href="/en/players/aq/">Aq</a>,
 <a href="/en/players/ar/">Ar</a>,
 <a href="/en/players/as/">As</a>,
 <a href="/en/players/at/">At</a>,
 <a href="/en/players/au/">Au</a>,
 <a href="/en/players/aw/">Aw</a>,
 <a href="/en/players/ax/">Ax</a>,
 <a href="/en/players/ay/">Ay</a>,
 <a href="/en/players/az/">Az</a>,
 <a href="/en/players/ba/">Ba</a>,
 <a href="/en/players/bb/">Bb</a>,
 <a href="/en/players/be/">Be</a>,
 <a href="/en/player

In [304]:
url_list = ['https://fbref.com'+x['href'] for x in a_list]

In [305]:
url_list

['https://fbref.com/en/players/aa/',
 'https://fbref.com/en/players/ab/',
 'https://fbref.com/en/players/ac/',
 'https://fbref.com/en/players/ad/',
 'https://fbref.com/en/players/ae/',
 'https://fbref.com/en/players/af/',
 'https://fbref.com/en/players/ag/',
 'https://fbref.com/en/players/ah/',
 'https://fbref.com/en/players/ai/',
 'https://fbref.com/en/players/aj/',
 'https://fbref.com/en/players/ak/',
 'https://fbref.com/en/players/al/',
 'https://fbref.com/en/players/am/',
 'https://fbref.com/en/players/an/',
 'https://fbref.com/en/players/ao/',
 'https://fbref.com/en/players/ap/',
 'https://fbref.com/en/players/aq/',
 'https://fbref.com/en/players/ar/',
 'https://fbref.com/en/players/as/',
 'https://fbref.com/en/players/at/',
 'https://fbref.com/en/players/au/',
 'https://fbref.com/en/players/aw/',
 'https://fbref.com/en/players/ax/',
 'https://fbref.com/en/players/ay/',
 'https://fbref.com/en/players/az/',
 'https://fbref.com/en/players/ba/',
 'https://fbref.com/en/players/bb/',
 